<a href="https://colab.research.google.com/github/reeveboy/Simple-Information-Retrieval/blob/main/Simple_Information_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
!pip install rank-bm25

In [39]:
!git clone https://github.com/usnistgov/trec_eval.git
!make -C trec_eval
# Download cranqrel.trec.txt file
!wget https://raw.githubusercontent.com/oussbenk/cranfield-trec-dataset/main/cranqrel.trec.txt

fatal: destination path 'trec_eval' already exists and is not an empty directory.
make: Entering directory '/content/trec_eval'
make: 'trec_eval' is up to date.
make: Leaving directory '/content/trec_eval'
--2024-03-18 10:12:23--  https://raw.githubusercontent.com/oussbenk/cranfield-trec-dataset/main/cranqrel.trec.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23217 (23K) [text/plain]
Saving to: ‘cranqrel.trec.txt.1’

cranqrel.trec.txt.1 100%[===================>]  22.67K  --.-KB/s    in 0s      

2024-03-18 10:12:23 (110 MB/s) - ‘cranqrel.trec.txt.1’ saved [23217/23217]



In [40]:
# loading basic packages
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from nltk import word_tokenize
import re
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math
from collections import Counter
import subprocess
from sklearn.model_selection import ParameterGrid
from rank_bm25 import BM25Okapi

In [42]:
df = pd.read_xml("/content/drive/MyDrive/datasets/Mechanics of Search/cran.all.1400.xml")
df = df[~df['text'].isna()] # remove null data

queries = pd.read_xml("/content/drive/MyDrive/datasets/Mechanics of Search/cran.qry.xml")

In [43]:
from nltk.corpus import stopwords as nltk_en_stopwords
from spacy.lang.en import stop_words as spacy_en_stopwords
NLTK_EN = set(nltk_en_stopwords.words('english'))
SPACY_EN = spacy_en_stopwords.STOP_WORDS
stop_words = NLTK_EN.union(SPACY_EN)

Preprocessing the corpus

In [44]:
def preprocess(doc_text):
  text = doc_text.lower()
  text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text) # Removes special characteres

  tokens = word_tokenize(text) # tokenize text

  text = [words for words in tokens if words not in stop_words] # remove stop words

  ps = nltk.stem.PorterStemmer()
  stemmed = [ps.stem(words) for words in text] # Stem words to the root form

  return stemmed

In [45]:
df.columns

Index(['docno', 'title', 'author', 'bib', 'text'], dtype='object')

In [46]:
corpus = [' '.join([row['title'] or '', row['text'] or '']) for index, row in df.iterrows()]
processed_corpus = [preprocess(doc) for doc in corpus]

### Vector Space Model

In [47]:
class VSM:
  def __init__(self):
    self.corpus = [' '.join(doc) for doc in processed_corpus]
    self.vectorizer = TfidfVectorizer()
    self.documents_vector = self.vectorizer.fit_transform(self.corpus)

  def rank(self, query, top_n=100):
    q = ' '.join(preprocess(query))
    query_vector = self.vectorizer.transform([q])

    cosineSimilarities = cosine_similarity(self.documents_vector, query_vector).flatten() # Calculate cosine similarities
    related_docs_indices = cosineSimilarities.argsort()[::-1][:top_n]  # Get top_n indices

    ranked_results = []
    for rank, i in enumerate(related_docs_indices, start=1):
        doc_id = df.iloc[i]['docno']
        similarity_score = cosineSimilarities[i]
        ranked_results.append((doc_id, similarity_score))

    return ranked_results

### BM25 Model



In [48]:
class BM25:
  def __init__(self, k1, b):
    self.corpus = processed_corpus
    self.k1 = k1
    self.b = b
    self.avgdl = sum(len(doc) for doc in self.corpus) / len(self.corpus)
    self.idf = self.calculate_idf()

  def calculate_idf(self):
    idf = {}
    doc_count = len(self.corpus)
    for doc in self.corpus:
      word_set = set(doc)
      for word in word_set:
        idf[word] = idf.get(word, 0) + 1

    for word, count in idf.items():
      idf[word] = math.log((doc_count - count + 0.5) / (count + 0.5) + 1)

    return idf

  def get_bm25_score(self, query: list[str], doc: list[str]) -> float:
    score = 0
    doc_counter = Counter(doc)
    doc_len = len(doc)
    for word in query:
      if word not in doc_counter:
        continue
      idf = self.idf.get(word, 0)
      f = doc_counter[word]
      score += idf * ((f * (self.k1 + 1)) / (f + self.k1 * (1 - self.b + self.b * (doc_len / self.avgdl))))
    return score

  def rank(self, query: str, top_n: int = 100) -> list[tuple[str, float]]:
    q = preprocess(query)  # You should define preprocess function
    scores = []

    for i, doc in enumerate(self.corpus):
      score = self.get_bm25_score(q, doc)
      scores.append((i, score))

    scores.sort(key=lambda x: x[1], reverse=True)

    ranked_results = []
    for rank, (idx, score) in enumerate(scores[:top_n], start=1):
      doc_id = df.iloc[idx]['docno']  # Assuming df is defined somewhere
      similarity_score = score
      ranked_results.append((doc_id, similarity_score))

    return ranked_results

### Relevance Model

In [49]:
class RelevanceModel:
  def __init__(self, mu, lambda_, top_n, top_terms):
    self.corpus = [' '.join(doc) for doc in processed_corpus]
    self.mu = mu
    self.lambda_ = lambda_
    self.top_n = top_n
    self.top_terms = top_terms
    self.vectorizer = TfidfVectorizer()
    self.documents_tfidf = self.vectorizer.fit_transform(self.corpus)
    self.vocab = np.array(self.vectorizer.get_feature_names_out())

  def rank(self, query, n=100):
    query = ' '.join(preprocess(query))

    # Initial ranking
    initial_scores = self.rank_documents(query)
    top_docs = [idx for idx, _ in initial_scores[:self.top_n]]  # Select top documents

    # Feedback query expansion using top documents
    relevant_docs_tfidf = self.documents_tfidf[top_docs]
    new_query = self.expand_query(query, relevant_docs_tfidf)

    # Re-rank documents using the new query
    re_ranked_scores = self.rank_documents(new_query)

    ranked_results = []
    for idx, score in re_ranked_scores[:n]:
      doc_id = df.iloc[idx]['docno']
      similarity_score = score
      ranked_results.append((doc_id, similarity_score))

    return ranked_results

  def rank_documents(self, query):
    query_tfidf = self.vectorizer.transform([query])
    similarities = cosine_similarity(query_tfidf, self.documents_tfidf)
    similarities = similarities.flatten()
    document_scores = [(i, score) for i, score in enumerate(similarities)]
    document_scores.sort(key=lambda x: x[1], reverse=True)
    return document_scores

  def expand_query(self, original_query, relevant_docs_tfidf):
    # Calculate query likelihood P(t | R) for each term
    query_tfidf = self.vectorizer.transform([original_query])
    doc_length = relevant_docs_tfidf.sum(axis=1)
    doc_length_norm = doc_length / (doc_length + self.mu)
    query_likelihood = np.asarray(relevant_docs_tfidf.sum(axis=0) / doc_length_norm).flatten()

    # Get top terms with higher P(t | R)
    top_term_indices = np.argsort(query_likelihood)[::-1][:self.top_terms]
    top_term_indices = top_term_indices[top_term_indices < len(self.vocab)]
    top_terms = self.vocab[top_term_indices]

    # Interpolate with original query
    original_query_tfidf = self.vectorizer.transform([original_query])
    new_query_tfidf = self.lambda_ * query_likelihood[top_term_indices] + (1 - self.lambda_) * original_query_tfidf[:, top_term_indices]

    # Construct new query
    new_query = " ".join(top_terms)
    return new_query

In [50]:
# Helper function to output results to files
def output_results_to_trec_format(results, model_name, output_file):
    with open(output_file, 'w') as f:
        for query_id, docs in results.items():
            for rank, (doc_id, score) in enumerate(docs, start=1):
                line = f"{query_id} 0 {doc_id} {rank} {score} {model_name}\n"
                f.write(line)

In [89]:
vsm_model = VSM()
bm25_model = BM25(k1=1.0, b=0.7)
relevance_model = RelevanceModel(mu=100, lambda_=0.6, top_n=10, top_terms=35)

# Dictionary to store results for each query
vsm_results = {}
bm25_results = {}
relevance_results = {}

# Execute VSM model for each query
for query_info in queries.to_dict(orient='records'):
  query_num = query_info['num']
  query_title = query_info["title"]
  vsm_result = vsm_model.rank(query_title, top_n=100)  # Execute VSM model for the query
  vsm_results[query_num] = vsm_result

  bm25_result = bm25_model.rank(query_title, top_n=100)  # Execute BM25 model for the query
  bm25_results[query_num] = bm25_result

  relevance_result = relevance_model.rank(query_title, n=100)  # Execute RelevanceModel for the query
  relevance_results[query_num] = relevance_result

!rm vsm_results.txt
!rm bm25_results.txt
!rm relevance_results.txt

# Output VSM results to file in TREC format
output_results_to_trec_format(vsm_results, "vsm", "vsm_results.txt")
output_results_to_trec_format(bm25_results, "bm25", "bm25_results.txt")
output_results_to_trec_format(relevance_results, "relevance", "relevance_results.txt")

In [52]:
# Helper function to run trec_eval command
def run_trec_eval_command(trec_eval_path, qrel_file, results_file):
  command = [trec_eval_path, "-m", "P.5", "-m", "ndcg", "-m", "map", qrel_file, results_file]
  try:
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output, error = process.communicate()

    output_str = output.decode("utf-8")
    output_lines = output_str.split("\n")
    metric_values = {}
    for line in output_lines:
      if line.strip() == "":
        continue
      parts = line.split()
      if len(parts) < 3:
        print("Unexpected output format:", line)
        continue
      metric = parts[0]
      value = float(parts[2])
      metric_values[metric] = value

    return metric_values
  except Exception as e:
    print("Error running trec_eval:", e)
    return {}


In [90]:
trec_eval_path = "./trec_eval/trec_eval"
path_to_cranfield_qrel = "/content/cranqrel.trec.txt"
path_to_vsm_results = "/content/vsm_results.txt"
path_to_bm25_results = "/content/bm25_results.txt"
path_to_relevance_results = "/content/relevance_results.txt"

vsm_score = run_trec_eval_command(trec_eval_path, path_to_cranfield_qrel, path_to_vsm_results)
bm_score = run_trec_eval_command(trec_eval_path, path_to_cranfield_qrel, path_to_bm25_results)
relevance_score = run_trec_eval_command(trec_eval_path, path_to_cranfield_qrel, path_to_relevance_results)
print(f"VSM Results: {vsm_score}")
print(f"BM25 Results: {bm_score}")
print(f"RM Results: {relevance_score}")

VSM Results: {'map': 0.0105, 'P_5': 0.0158, 'ndcg': 0.0408}
BM25 Results: {'map': 0.0094, 'P_5': 0.0145, 'ndcg': 0.0376}
RM Results: {'map': 0.0138, 'P_5': 0.0211, 'ndcg': 0.0417}


## Comparing My BM25 algorithm against BM25Rank module

In [56]:
class BM25Rank:
  def __init__(self):
    self.tokenized_corpus = processed_corpus
    self.bm25 = BM25Okapi(processed_corpus)

  def rank(self, query, top_n=100):
    tokenized_query = query.split()
    doc_scores = self.bm25.get_scores(tokenized_query)
    ranked_indices = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)[:top_n]

    ranked_results = []
    for idx in ranked_indices:
      doc_id = idx  # Assuming doc_id is just the index
      similarity_score = doc_scores[idx]
      ranked_results.append((doc_id, similarity_score))

    return ranked_results

In [86]:
my_bm25_results = {}
bm25_results = {}

bm25 = BM25Rank()
my_bm25 = BM25(1.0, 0.7)
for query_info in queries.to_dict(orient='records'):
    query_num = query_info['num']
    query_title = query_info["title"]

    bm25_result = bm25_model.rank(query_title, top_n=100)  # Execute BM25 model for the query
    bm25_results[query_num] = bm25_result

    my_bm25_result = my_bm25.rank(query_title, top_n=100)  # Execute BM25 model for the query
    my_bm25_results[query_num] = my_bm25_result

# Output results to a file in trec_eval format
output_results_to_trec_format(bm25_results, "bm25", "module.txt")
output_results_to_trec_format(bm25_results, "bm25", "my.txt")

In [87]:
# Evaluate the new parameters
score = run_trec_eval_command(trec_eval_path, path_to_cranfield_qrel, "/content/module.txt")
print(f"Module score: {score}")

Module score: {'map': 0.0089, 'P_5': 0.0145, 'ndcg': 0.0384}


In [88]:
my_score = run_trec_eval_command(trec_eval_path, path_to_cranfield_qrel, "/content/my.txt")
print(f"My score: {my_score}")

My score: {'map': 0.0089, 'P_5': 0.0145, 'ndcg': 0.0384}


Finding the best hyperparameters for BM25

In [85]:
# Define a grid of hyperparameters to search through
param_grid = {
  'k1': [0.5, 1.0, 1.2, 1.5, 1.7, 1.9, 2.2],
  'b': [0.3, 0.5, 0.7, 0.75, 0.8, 0.9]
}

metrics = ["P_5", "ndcg", "map"]

# Previous best parameters
best_k1 = 1.2
best_b = 0.75
best_param = (best_k1, best_b)

# Placeholder for best results
best_results = {metric: -1 for metric in metrics}

for params in ParameterGrid(param_grid):
  bm25_model = BM25(k1=params['k1'], b=params['b'])

  # Execute BM25 model for each query
  bm25_results = {}
  for query_info in queries.to_dict(orient='records'):
    query_num = query_info['num']
    query_title = query_info["title"]
    bm25_result = bm25_model.rank(query_title, top_n=100)  # Execute BM25 model for the query
    bm25_results[query_num] = bm25_result

  # Output results to a file in trec_eval format
  output_results_to_trec_format(bm25_results, "bm25", "bm25_results.txt")

  # Evaluate the model and get the average score
  metrics = ["P_5", "ndcg", "map"]  # Corrected "P.5" to "P_5"
  path_to_cranfield_qrel = "/content/cranqrel.trec.txt"
  path_to_bm25_results = "/content/bm25_results.txt"

  # Evaluate the new parameters
  new_results = run_trec_eval_command(trec_eval_path, path_to_cranfield_qrel, path_to_bm25_results)

  # Compare the results
  better_than_best = False
  c = 0
  for metric in metrics:
    if metric not in new_results:
      print("Metrics not present")
      continue
    if new_results[metric] >= best_results[metric]:
      c += 1

  if c == 3:
    better_than_best = True

  # If the new parameters are better, update bestparam
  if better_than_best:
    best_k1, best_b = params['k1'], params['b']
    best_param = (best_k1, best_b)
    print("New parameters are better. Updated bestparam:", best_param)
    # Update best_results
    best_results = new_results
  else:
    print(f"New parameters {(params['k1'], params['b'])} are not better. Keeping current bestparam: {best_param}")


New parameters are better. Updated bestparam: (0.5, 0.3)
New parameters are better. Updated bestparam: (1.0, 0.3)
New parameters are better. Updated bestparam: (1.2, 0.3)
New parameters (1.5, 0.3) are not better. Keeping current bestparam: (1.2, 0.3)
New parameters (1.7, 0.3) are not better. Keeping current bestparam: (1.2, 0.3)
New parameters (1.9, 0.3) are not better. Keeping current bestparam: (1.2, 0.3)
New parameters (2.2, 0.3) are not better. Keeping current bestparam: (1.2, 0.3)
New parameters (0.5, 0.5) are not better. Keeping current bestparam: (1.2, 0.3)
New parameters are better. Updated bestparam: (1.0, 0.5)
New parameters (1.2, 0.5) are not better. Keeping current bestparam: (1.0, 0.5)
New parameters (1.5, 0.5) are not better. Keeping current bestparam: (1.0, 0.5)
New parameters (1.7, 0.5) are not better. Keeping current bestparam: (1.0, 0.5)
New parameters (1.9, 0.5) are not better. Keeping current bestparam: (1.0, 0.5)
New parameters (2.2, 0.5) are not better. Keeping cu

Finding the best hyperparameters for Relevance Model

In [84]:
# Define a grid of hyperparameters to search through
param_grid = {
  'mu': [100, 200, 350, 500, 600],
  'lambda_': [0.2, 0.4, 0.6],
  'top_n': [5, 10, 15],
  'top_terms': [25, 30, 35]
}

metrics = ["P_5", "ndcg", "map"]

# Previous best parameters
best_mu = 500
best_lambda = 0.5
best_top_n = 10
best_top_terms = 20
best_params = (best_mu, best_lambda, best_top_n, best_top_terms)

# Placeholder for best results
best_results = {metric: -1 for metric in metrics}

for params in ParameterGrid(param_grid):
  relevance_model = RelevanceModel(mu=params['mu'], lambda_=params['lambda_'],
                                    top_n=params['top_n'], top_terms=params['top_terms'])

  # Execute Relevance Model for each query
  relevance_results = {}
  for query_info in queries.to_dict(orient='records'):
    query_num = query_info['num']
    query_title = query_info["title"]
    relevance_result = relevance_model.rank(query_title, n=100)  # Execute RM for the query
    relevance_results[query_num] = relevance_result

  # Output results to a file in trec_eval format
  output_results_to_trec_format(relevance_results, "relevance", "relevance_results.txt")

  # Evaluate the model and get the average score
  path_to_cranfield_qrel = "/content/cranqrel.trec.txt"
  path_to_relevance_results = "/content/relevance_results.txt"

  # Evaluate the new parameters
  new_results = run_trec_eval_command(trec_eval_path, path_to_cranfield_qrel, path_to_relevance_results)

  # Compare the results
  better_than_best = False
  c = 0
  for metric in metrics:
    if metric not in new_results:
      print(new_results)
      continue
    if new_results[metric] >= best_results[metric]:
      c += 1

  if c == 3:
    better_than_best = True

  # If the new parameters are better, update best_params
  if better_than_best:
    best_mu, best_lambda, best_top_n, best_top_terms = params['mu'], params['lambda_'], params['top_n'], params['top_terms']
    best_params = (best_mu, best_lambda, best_top_n, best_top_terms)
    print("New parameters are better. Updated best_params:", best_params)
    print(f"old: {best_results}, new: {new_results}")
    print("=================================================")
    # Update best_results
    best_results = new_results
  else:
    print(f"New parameters {(params['mu'], params['lambda_'], params['top_n'], params['top_terms'])} are not better. Keeping current bestparam: {best_params}")
    print(f"old: {best_results}, new: {new_results}")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

New parameters are better. Updated best_params: (100, 0.2, 5, 25)
old: {'P_5': -1, 'ndcg': -1, 'map': -1}, new: {'map': 0.0114, 'P_5': 0.0197, 'ndcg': 0.0394}
New parameters (100, 0.2, 5, 30) are not better. Keeping current bestparam: (100, 0.2, 5, 25)
old: {'map': 0.0114, 'P_5': 0.0197, 'ndcg': 0.0394}, new: {'map': 0.0114, 'P_5': 0.0184, 'ndcg': 0.0394}
+++++++++++++++++++++++++++++++++++++++++++++++++
New parameters (100, 0.2, 5, 35) are not better. Keeping current bestparam: (100, 0.2, 5, 25)
old: {'map': 0.0114, 'P_5': 0.0197, 'ndcg': 0.0394}, new: {'map': 0.0097, 'P_5': 0.0145, 'ndcg': 0.0386}
+++++++++++++++++++++++++++++++++++++++++++++++++
New parameters (100, 0.2, 10, 25) are not better. Keeping current bestparam: (100, 0.2, 5, 25)
old: {'map': 0.0114, 'P_5': 0.0197, 'ndcg': 0.0394}, new: {'map': 0.0125, 'P_5': 0.0197, 'ndcg': 0.0385}
+++++++++++++++++++++++++++++++++++++++++++++++++
New parameters are better. Updated best_params: (100, 0.2, 10, 30)
old: {'map': 0.0114, 'P_5'